In [1]:
import pandas as pd
import numpy as np
# import pickle

import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
from d2v_recommender import *
from config import config
from src.processing import load_d2v_formated_data
from src.d2v_recommender import D2V_Recommender


In [2]:
recommender = D2V_Recommender(**config.d2v_params)
recommender.load_rated_vec(config.rated_embeddings_path)
_ = recommender.load_rater_vec(config.rater_embeddings_path)

INFO:gensim.utils:loading Word2VecKeyedVectors object from /root/notebooks/../data/dev/models/rated.vectors
INFO:gensim.utils:setting ignored attribute vectors_norm to None
INFO:gensim.utils:loaded /root/notebooks/../data/dev/models/rated.vectors


In [3]:
train_ = pd.read_csv(config.train_data_path, nrows=1000)
test_ = pd.read_csv(config.test_data_path, nrows=1000)
print(train_.shape)
train_.head() 

(1000, 4)


,rater,rated,r,m
0,393,116083,8,0.0
1,393,5962,9,0.0
2,393,24614,1,0.0
3,393,104576,5,0.0
4,393,181022,7,0.0


In [4]:

train_["set"] = "train"
test_["set"] = "test"
data = pd.concat([train_, test_])  # we will ignore the index
print("concatenated")

print(f"Train N={len(train_)} - Test N={len(test_)}")
assert len(data) == (len(train_) + len(test_))
a = data["rater"].progress_apply(recommender.get_single_rater_vec)
b = data["rated"].progress_apply(recommender.get_single_rated_vec)
# TODO: could be even further vectorized
data["vec_delta"] = a - b  # piecewise array operations

 46%|████▌     | 922/2000 [00:00<00:00, 9214.21it/s]concatenated
Train N=1000 - Test N=1000
100%|██████████| 2000/2000 [00:00<00:00, 172480.89it/s]


In [12]:
def get_vec_diff(rater_rated):
    rater, rated = rater_rated
    vec1 = recommender.get_single_rater_vec(rater)
    vec2 = recommender.get_single_rated_vec(rated)
    if vec2 is not None:
        return (vec1 - vec2)[0]
    else:
        return None

train_[["rater","rated"]].apply(lambda x: get_vec_diff(x), axis=1)

0                                                   None
1      [-0.025143957, 0.038365867, 0.08811399, 0.0415...
2                                                   None
3      [-0.02080357, 0.04270625, 0.09245438, 0.045902...
4                                                   None
                             ...                        
995                                                 None
996    [-0.0050131753, 0.017379366, 0.034791254, 0.01...
997    [-0.010202071, 0.0121904705, 0.02960236, 0.011...
998    [-0.008433124, 0.013959417, 0.031371307, 0.013...
999    [-0.015170673, 0.0072218687, 0.024633758, 0.00...
Length: 1000, dtype: object